In [55]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
import os
import sys
from pyspark.sql.functions import *
import psycopg2

In [56]:
# set Java home
os.environ["JAVA_HOME"] = "C:/Users/User/AppData/Local/Programs/Eclipse Adoptium/jdk-11.0.25.9-hotspot"

In [57]:
conf = SparkConf() \
    .setAppName("ETLPipeline") \
    .setMaster("local") \
    .set("spark.driver.extraClassPath","C:/jars/*")

In [58]:
sc = SparkContext.getOrCreate(conf=conf)
etl = SparkSession(sc)

In [59]:
sc, etl

(<SparkContext master=local appName=ETLPipeline>,
 <pyspark.sql.session.SparkSession at 0x1f7772277d0>)

In [60]:
#get password from environmnet var
pwd = "demopass"
uid = "etl"
#sql db details
server = "localhost"
src_db = "Diablo"
target_db = "test_db"
src_driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
target_driver = "org.postgresql.Driver"

In [61]:
# source connection
src_url = f"jdbc:sqlserver://{server}:1433;databaseName={src_db};user={uid};password={pwd};encrypt=false;"

# target connection
target_url = f"jdbc:postgresql://{server}:5432/{target_db}?user={uid}&password={pwd}"

In [62]:
sql = """select  t.name as table_name from sys.tables t 
where t.name in ('Games','Items','Users','GameTypes') """

In [63]:
# Let's test our connection
dfs=etl.read. \
    format("jdbc"). \
    options(driver=src_driver, user=uid, password=pwd, url=src_url, query=sql). \
    load()
dfs.show()

+----------+
|table_name|
+----------+
|     Games|
| GameTypes|
|     Items|
|     Users|
+----------+



In [64]:
data_collect = dfs.collect()
# looping thorough each row of the dataframe
for row in data_collect:
    # while looping through each
    # row printing the data of table_name
    print(row["table_name"])

Games
GameTypes
Items
Users


In [65]:
def extract():
    try:
        dfs=etl.read. \
            format("jdbc"). \
            options(driver=src_driver,user=uid, password=pwd,url=src_url,query=sql). \
            load()
        # get table names
        data_collect = dfs.collect()
        # looping thorough each row of the dataframe
        for row in data_collect:
        # while looping through each
        # row printing the data of table_name
            print(row["table_name"])
            tbl_name = row["table_name"]
            df = etl.read \
            .format("jdbc") \
            .option("driver", src_driver) \
            .option("user", uid) \
            .option("password", pwd) \
            .option("url", src_url) \
            .option("dbtable", f"dbo.{tbl_name}") \
            .load()
            #print(df.show(10))
            load(df, tbl_name)
            print("Data loaded successfully")
    except Exception as e:
        print("Data extract error: " + str(e))

In [66]:
def load(df, tbl):
    try:
        rows_imported = 0
        print(f'importing rows {rows_imported} to {rows_imported + df.count()}... for table {tbl}')
        df.write.mode("overwrite") \
        .format("jdbc") \
        .option("url", target_url) \
        .option("user", uid) \
        .option("password", pwd) \
        .option("driver", target_driver) \
        .option("dbtable", "src_" + tbl) \
        .save()
        print("Data imported successful")
        rows_imported += df.count()
    except Exception as e:
        print("Data load error: " + str(e))

In [67]:
# Function Call
extract()

Games
importing rows 0 to 241... for table Games
Data imported successful
Data loaded successfully
GameTypes
importing rows 0 to 5... for table GameTypes
Data imported successful
Data loaded successfully
Items
importing rows 0 to 577... for table Items
Data imported successful
Data loaded successfully
Users
importing rows 0 to 71... for table Users
Data imported successful
Data loaded successfully
